In [1]:
import os
import numpy as np
import efficientnet.tfkeras
from tensorflow.keras.models import load_model
from tensorflow.keras import models
import tensorflow as tf
from focal_loss import BinaryFocalLoss

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]=""

In [3]:
model_dir ='/media/tohn/SSD/ModelTrainByImages/R2_5/models/B5R2b5_5FP_1FC_fold5_1.h5'
model = load_model(model_dir)
height = width = model.input_shape[1]

x = model.get_layer('head_pooling').output
prediction_layer = model.output
model2 = models.Model(inputs= model.input, outputs=[prediction_layer,x]) 

In [4]:
#ไม่ต้องเปลี่ยนก็ได้ เพราะเอา Label มาใช้เฉยๆ 
# validation
import pandas as pd
base_dir  = '/media/tohn/SSD/Images/Image4/validation/'
dataframe = pd.read_csv( '/home/yupaporn/codes/USAI/Validationdf_fold4.csv')
validation_dir = os.path.join(base_dir, 'validation')

#Train
train_df = pd.read_csv( '/home/yupaporn/codes/USAI/Traindf_fold4.csv')
base_dir0 = '/media/tohn/SSD/Images/Image1/'
os.chdir(base_dir0)
train_dir = os.path.join(base_dir0, 'train')

In [5]:
import pandas as pd
df0 = pd.read_csv (r'/home/yupaporn/codes/USAI/Validationdf_fold5.csv')
print(df0 .shape)
dataframe = df0[(df0['Path Crop']!='None' )&(df0['Path Crop']!='Nan')]
# a = df01[df01['Case'].between(1, 10)]
# dataframe = df01[df01['Abs Position']!='P8']
print(dataframe.shape)
print('Normal: ',dataframe[dataframe['Class']=='Normal'].shape)
print('Abnormal: ',dataframe[dataframe['Class']=='Abnormal'].shape)
dataframe.head(5)

(656, 25)
(656, 25)
Normal:  (429, 25)
Abnormal:  (227, 25)


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Case,Abs Position,Sub Position,Class,Sub_class,Path Full,...,left,top,width,height,Rleft,Rtop,Rwidth,Rheight,FileName,filename
0,0,0,34,34,42,P5,P52,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,94.0,80.0,572.0,585.0,0.104950,0.089112,0.633663,0.651394,AB01 P5-2 C042.JPG,AB01 P5-2 C042.JPG
1,1,1,35,35,42,P1,P1,Abnormal,AB02,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,231.0,134.0,544.0,380.0,0.229956,0.149621,0.541516,0.422222,AB02 P1 C042.JPG,AB02 P1 C042.JPG
2,2,2,36,36,42,P2,P2,Abnormal,AB02,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,238.0,140.0,497.0,549.0,0.237177,0.155682,0.494585,0.610101,AB02 P2 C042.JPG,AB02 P2 C042.JPG
3,3,3,37,37,42,P5,P51,Abnormal,AB02,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,318.0,160.0,494.0,601.0,0.318902,0.177904,0.494545,0.668687,AB02 P5-1 C042.JPG,AB02 P5-1 C042.JPG
4,4,4,38,38,42,P4,P42,Abnormal,AB02,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,300.0,143.0,474.0,621.0,0.302369,0.159722,0.477148,0.690909,AB02 P4-2 C042.JPG,AB02 P4-2 C042.JPG


In [6]:
# dataframe.info()

In [6]:
batch_size = 64
epochs = 10

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      brightness_range=[0.5,1.5],
      shear_range=0.4,
      zoom_range=0.2,
      horizontal_flip=False,
      fill_mode='nearest')

train_generator = train_datagen.flow_from_dataframe(
        dataframe = dataframe,
        directory = train_dir,
        x_col = 'Path Crop',
        y_col = 'Views',
        target_size = (height, width),
        batch_size=batch_size,
        color_mode= 'rgb',
        class_mode='categorical')

#label
labels = (train_generator.class_indices)
labels = dict((v,k.replace("C","")) for k,v in labels.items())
print(labels)

Found 656 validated image filenames belonging to 5 classes.
{0: 'FP-A', 1: 'FP-B', 2: 'FP-', 3: 'FP-D', 4: 'FP-E'}


In [7]:
from tensorflow.keras.preprocessing import image
def predict_image(img_path):
    # Read the image and resize it
    img = image.load_img(img_path, target_size=(height, width))
    # Convert it to a Numpy array with target shape.
    x = image.img_to_array(img)
    # Reshape
    x = x.reshape((1,) + x.shape)
    x /= 255.
    result = model2.predict([x])
    result1 = result[0][0]
    result2 = result[1][0]

    return result1, result2

#Predict
pred_list = list()
prob_list = list()
vectorlist = list()
img_path=dataframe['Path Crop'].tolist()
for i in range(0,len(img_path)):
    result1,result2 = predict_image(img_path[i])
    result = np.argmax(result1)
    pred_list.append(labels[result])
    prob_list.append(result1[result])
    vectorlist.append(result2.tolist())
    
dataframe['category'] = pred_list
dataframe['Prob'] = prob_list

In [8]:
vector_array = np.array(vectorlist)
vector_array

array([[-0.18752854, -0.14149643,  0.05940841, ..., -0.14756207,
         0.03294284, -0.1608839 ],
       [ 0.17435493,  0.14887179,  0.13162172, ..., -0.1152407 ,
        -0.19928657,  0.96768284],
       [ 0.0676467 ,  0.29701239,  0.81593567, ..., -0.13582343,
        -0.19947474,  1.03704822],
       ...,
       [ 0.49436685, -0.07556017,  0.02149931, ...,  0.02723564,
        -0.09507556,  0.34695199],
       [ 0.45816442, -0.03971249,  0.24405265, ..., -0.12169399,
        -0.1634376 ,  0.53478068],
       [-0.01313063, -0.0599184 , -0.18049052, ...,  0.00511629,
        -0.17048031,  0.19936305]])

In [9]:
dataset = pd.DataFrame(vector_array[:, :])
dataset

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,-0.187529,-0.141496,0.059408,-0.085317,-0.061441,-0.036807,0.361733,-0.046993,-0.026996,-0.164707,...,-0.166111,0.153442,0.045952,-0.122765,0.081446,-0.164327,-0.105913,-0.147562,0.032943,-0.160884
1,0.174355,0.148872,0.131622,0.119818,0.453863,0.242727,0.298383,0.111581,-0.144613,-0.144121,...,-0.146352,-0.111524,-0.201977,-0.125758,0.225444,0.457006,0.005407,-0.115241,-0.199287,0.967683
2,0.067647,0.297012,0.815936,0.138661,0.242373,0.598718,0.950902,0.299606,-0.013170,-0.188022,...,-0.167212,-0.101260,-0.205910,-0.058964,-0.093067,0.380495,0.460630,-0.135823,-0.199475,1.037048
3,-0.172844,-0.129973,0.161890,0.097445,-0.163525,-0.165896,0.385491,-0.174794,0.039391,-0.135886,...,-0.181151,0.117585,0.403158,-0.105619,0.404829,-0.187930,-0.178770,-0.104841,0.203931,-0.154793
4,-0.201331,-0.025423,0.562680,0.428355,0.057749,0.021166,0.967933,-0.113979,0.120943,-0.158807,...,-0.142855,0.325628,0.462462,-0.102096,0.426077,-0.181371,-0.193593,-0.017496,-0.076928,-0.160144
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
651,-0.192995,-0.196337,-0.134284,0.098834,-0.152699,-0.172450,-0.032410,-0.081889,0.132783,0.323654,...,-0.152894,0.589930,0.546163,-0.003418,0.577732,-0.207290,-0.205079,-0.105243,0.176994,-0.157321
652,-0.162739,-0.171227,-0.048529,0.323934,0.082326,-0.216421,0.214099,-0.145684,0.394911,-0.238300,...,-0.205419,0.094257,0.012855,-0.163659,0.473510,-0.201952,-0.066799,-0.179759,-0.088749,-0.150173
653,0.494367,-0.075560,0.021499,0.104018,-0.133141,0.179355,0.302191,-0.026367,-0.189166,-0.105261,...,0.526303,0.834761,-0.173232,0.494912,0.270599,-0.030763,-0.022033,0.027236,-0.095076,0.346952
654,0.458164,-0.039712,0.244053,0.478749,-0.150483,0.071534,0.444147,-0.230274,-0.100825,-0.210610,...,0.560108,0.775162,-0.098771,0.277773,0.715069,-0.045138,-0.077578,-0.121694,-0.163438,0.534781


In [10]:
a = dataframe['Class'].tolist()
b = dataframe['Sub Position'].tolist()
c = dataframe['Views'].tolist()
d = dataframe['Sub_class'].tolist()
dataset['Class'] = a 
dataset['SubPosition'] = b
dataset['Views'] = c
dataset['Sub_class'] = d

In [11]:
dataset

,0,1,2,3,4,5,6,7,8,9,...,2042,2043,2044,2045,2046,2047,Class,SubPosition,Views,Sub_class
0,-0.187529,-0.141496,0.059408,-0.085317,-0.061441,-0.036807,0.361733,-0.046993,-0.026996,-0.164707,...,0.081446,-0.164327,-0.105913,-0.147562,0.032943,-0.160884,Abnormal,P52,FP-C,AB01
1,0.174355,0.148872,0.131622,0.119818,0.453863,0.242727,0.298383,0.111581,-0.144613,-0.144121,...,0.225444,0.457006,0.005407,-0.115241,-0.199287,0.967683,Abnormal,P1,FP-A,AB02
2,0.067647,0.297012,0.815936,0.138661,0.242373,0.598718,0.950902,0.299606,-0.013170,-0.188022,...,-0.093067,0.380495,0.460630,-0.135823,-0.199475,1.037048,Abnormal,P2,FP-A,AB02
3,-0.172844,-0.129973,0.161890,0.097445,-0.163525,-0.165896,0.385491,-0.174794,0.039391,-0.135886,...,0.404829,-0.187930,-0.178770,-0.104841,0.203931,-0.154793,Abnormal,P51,FP-C,AB02
4,-0.201331,-0.025423,0.562680,0.428355,0.057749,0.021166,0.967933,-0.113979,0.120943,-0.158807,...,0.426077,-0.181371,-0.193593,-0.017496,-0.076928,-0.160144,Abnormal,P42,FP-B,AB02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
651,-0.192995,-0.196337,-0.134284,0.098834,-0.152699,-0.172450,-0.032410,-0.081889,0.132783,0.323654,...,0.577732,-0.207290,-0.205079,-0.105243,0.176994,-0.157321,Normal,P61,FP-C,Normal
652,-0.162739,-0.171227,-0.048529,0.323934,0.082326,-0.216421,0.214099,-0.145684,0.394911,-0.238300,...,0.473510,-0.201952,-0.066799,-0.179759,-0.088749,-0.150173,Normal,P62,FP-C,Normal
653,0.494367,-0.075560,0.021499,0.104018,-0.133141,0.179355,0.302191,-0.026367,-0.189166,-0.105261,...,0.270599,-0.030763,-0.022033,0.027236,-0.095076,0.346952,Normal,P71,FP-D,Normal
654,0.458164,-0.039712,0.244053,0.478749,-0.150483,0.071534,0.444147,-0.230274,-0.100825,-0.210610,...,0.715069,-0.045138,-0.077578,-0.121694,-0.163438,0.534781,Normal,P72,FP-D,Normal


In [12]:
os.chdir( '/home/yupaporn/codes/USAI/')
testdf = dataset
testdf.to_csv('FVvalidation_5FP_RF_5FP_fold5_1.csv')